In [47]:
import pickle

test_x = []
def get_feat(f):
    with open(f,'rb') as fin:
        a,b = pickle.load(fin)
        return b

feat_files = [
    '../features/glove_cnn_gru_4_feat.pkl',
    '../features/glove_cudnn_gru_4_feat.pkl',
    '../features/glove_lstm_v1_4_feat.pkl',
    '../features/glove_gru_v1_4_feat.pkl'
]
for f in feat_files:
    b = get_feat(f)
    test_x.append(b)

import numpy as np
test_x = np.array(test_x)
print(test_x.shape)
mean_test_x = np.mean(test_x,axis=0)
print(mean_test_x.shape)

(4, 153164, 6)
(153164, 6)


In [48]:
import pandas as pd
sample_submission = pd.read_csv("../input/sample_submission.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
sample_submission[list_classes] = mean_test_x
sample_submission.to_csv("../results/avg_bagging.csv.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

                 id     toxic  severe_toxic   obscene    threat    insult  \
0  00001cee341fdb12  0.995861      0.465433  0.965190  0.157851  0.916416   
1  0000247867823ef7  0.000473      0.000004  0.000121  0.000003  0.000058   
2  00013b17ad220c46  0.001627      0.000072  0.000693  0.000064  0.000392   
3  00017563c3f7919a  0.000332      0.000002  0.000076  0.000006  0.000047   
4  00017695ad8997eb  0.007078      0.000040  0.001054  0.000080  0.000575   

   identity_hate  
0       0.497567  
1       0.000016  
2       0.000145  
3       0.000009  
4       0.000124  
save done


In [51]:
def avg_res(res_files):
    test_x = []
    for f in res_files:
        test_x.append(pd.read_csv(f)[list_classes].values)
    return np.mean(test_x,axis=0)

res_fs = ['../results/pub9854.csv','../results/xgb9863_fold10.gz']
avg_sub = avg_res(res_fs)
sample_submission[list_classes] = avg_sub
sample_submission.to_csv("../results/avg_sub.csv.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

                 id     toxic  severe_toxic   obscene    threat    insult  \
0  00001cee341fdb12  0.992304      0.356113  0.970673  0.150726  0.919923   
1  0000247867823ef7  0.000966      0.000378  0.000531  0.000042  0.000588   
2  00013b17ad220c46  0.003808      0.000625  0.001785  0.000108  0.001243   
3  00017563c3f7919a  0.000648      0.000148  0.000315  0.000100  0.000398   
4  00017695ad8997eb  0.005718      0.000188  0.000903  0.000088  0.001192   

   identity_hate  
0       0.423459  
1       0.000230  
2       0.000392  
3       0.000075  
4       0.000149  
save done


In [ ]:
# def pow_avg_res(res_files):
#     test_x = []
#     for f in res_files:
#         test_x.append(pd.read_csv(f)[list_classes].values)
#     file_cnt = len(res_files)
#     base = test_x[0]
#     for j in range(file_cnt-1):
#         for i in range(6):
#             base[:,i] = base[:,i]*test_x[j+1][:,i]
#     base = base ** (1/file_cnt)
#     return base

# res_fs = ['../results/pub9854.csv','../results/xgb9856.gz']
# avg_sub = pow_avg_res(res_fs)
# sample_submission[list_classes] = avg_sub
# sample_submission.to_csv("../results/pow_avg_sub.csv.gz", index=False, compression='gzip')
# print(sample_submission.head())
# print('save done')

In [52]:
# play with weight
df1 = pd.read_csv('../results/pub9854.csv')[list_classes].values
df2 = pd.read_csv('../results/xgb9863_fold10.gz')[list_classes].values
play_res = df1*0.3+df2*0.7
sample_submission[list_classes] = play_res
sample_submission.to_csv("../results/weighted_avg_sub.csv.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

                 id     toxic  severe_toxic   obscene    threat    insult  \
0  00001cee341fdb12  0.996279      0.304595  0.972031  0.207249  0.911433   
1  0000247867823ef7  0.000465      0.000165  0.000256  0.000029  0.000257   
2  00013b17ad220c46  0.001803      0.000269  0.000865  0.000062  0.000702   
3  00017563c3f7919a  0.000388      0.000079  0.000181  0.000054  0.000205   
4  00017695ad8997eb  0.004606      0.000104  0.000629  0.000055  0.000759   

   identity_hate  
0       0.483341  
1       0.000113  
2       0.000266  
3       0.000058  
4       0.000102  
save done
